In [44]:
import pandas as pd

# Load the data
file_path = 'data/auto_data.csv'
auto_data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
auto_data.head()


,Unnamed: 0,price,highway-mpg,city-mpg,peak-rpm,horsepower,compression-ratio,stroke,bore,fuel-system,...,length,wheel-base,engine-location,drive-wheels,body-style,num-of-doors,aspiration,fuel-type,make,normalized-losses
0,0,13495.0,27,21,5000.0,111.0,9.0,2.68,3.47,mpfi,...,168.8,88.6,front,rwd,convertible,2.0,std,gas,alfa-romero,NaN
1,1,16500.0,27,21,5000.0,111.0,9.0,2.68,3.47,mpfi,...,168.8,88.6,front,rwd,convertible,2.0,std,gas,alfa-romero,NaN
2,2,16500.0,26,19,5000.0,154.0,9.0,3.47,2.68,mpfi,...,171.2,94.5,front,rwd,hatchback,2.0,std,gas,alfa-romero,NaN
3,3,13950.0,30,24,5500.0,102.0,10.0,3.40,3.19,mpfi,...,176.6,99.8,front,fwd,sedan,4.0,std,gas,audi,164.0
4,4,17450.0,22,18,5500.0,115.0,8.0,3.40,3.19,mpfi,...,176.6,99.4,front,4wd,sedan,4.0,std,gas,audi,164.0


### Here's a plan for preprocessing the data for machine learning:

#### Remove Unnecessary Columns: The column Unnamed: 0 seems like an index column and can be removed.
#### Missing Values: We need to check for missing values and decide how to handle them.
#### Categorical Data: Convert categorical features into a format that can be used by machine learning models.
#### Feature Scaling: Scale numerical features if necessary.
#### Feature Engineering: Create new features if it seems beneficial.

### The dataset has missing values in several columns: price, peak-rpm, horsepower, stroke, bore, num-of-doors, and normalized-losses. Since price is our target variable, rows with missing price values are not useful for training a predictive model and should be removed. For the other columns, we have a few options:

##### * Imputation: Fill in the missing values with a statistic like the mean, median, or mode. This is common for numerical columns.
##### * Remove Rows or Columns: If a column has a significant number of missing values, it might be better to remove it. Similarly, rows with missing values can also be removed, especially if they are few.
##### * Categorical Columns: For categorical columns like num-of-doors, we can fill missing values with the most frequent category.
#### Let's proceed as follows:

#### * Remove rows where price is missing.
#### * For other numerical columns with missing values (peak-rpm, horsepower, stroke, bore), we'll impute missing values with the median.
#### * For the num-of-doors column, we'll fill missing values with the most frequent category.
#### * We need to decide what to do with the normalized-losses column based on the proportion of missing values.

In [45]:
# Remove the 'Unnamed: 0' column
auto_data.drop('Unnamed: 0', axis=1, inplace=True)

# Check for missing values
missing_values = auto_data.isnull().sum()
missing_values[missing_values > 0]


price                 4
peak-rpm              2
horsepower            2
stroke                4
bore                  4
num-of-doors          2
normalized-losses    41
dtype: int64

In [46]:
# Remove rows where 'price' is missing
auto_data = auto_data.dropna(subset=['price'])

# Impute missing values for numerical columns with the median
for column in ['peak-rpm', 'horsepower', 'stroke', 'bore']:
    median_value = auto_data[column].median()
    auto_data[column].fillna(median_value, inplace=True)

# Fill missing values in 'num-of-doors' with the most frequent category
most_frequent = auto_data['num-of-doors'].mode()[0]
auto_data['num-of-doors'].fillna(most_frequent, inplace=True)

# Check the proportion of missing values in 'normalized-losses'
normalized_losses_missing_ratio = auto_data['normalized-losses'].isnull().mean()

normalized_losses_missing_ratio


0.18407960199004975

In [47]:
# Impute missing values in 'normalized-losses' with the median
median_value = auto_data['normalized-losses'].median()
auto_data['normalized-losses'].fillna(median_value, inplace=True)

# Check the dataset for any remaining missing values
remaining_missing_values = auto_data.isnull().sum().sum()
remaining_missing_values


0

In [48]:
# Feature 1: Engine Efficiency
auto_data['engine_efficiency'] = auto_data['horsepower'] / auto_data['engine-size']

# Feature 2: Combined Mileage
# Assuming highway mileage is 60% of the time and city mileage is 40% of the time
auto_data['combined_mpg'] = (auto_data['highway-mpg'] * 0.6 + auto_data['city-mpg'] * 0.4)

# Feature 3: Polynomial Feature (engine-size squared)
auto_data['engine_size_squared'] = auto_data['engine-size'] ** 2

# Feature 4: Binning horsepower into categories
bins = [0, 100, 200, max(auto_data['horsepower'])]
labels = ['Low', 'Medium', 'High']
auto_data['horsepower_bin'] = pd.cut(auto_data['horsepower'], bins=bins, labels=labels)
# Define a mapping from bin labels to ordinal numbers
ordinal_map = {'Low': 1, 'Medium': 2, 'High': 3}

# Apply the mapping to the binned column
auto_data['horsepower_ordinal'] = auto_data['horsepower_bin'].map(ordinal_map)


# Feature 5: Interaction Term (engine-size * num-of-cylinders)
auto_data['engine_cylinders_interaction'] = auto_data['engine-size'] * auto_data['num-of-cylinders']

# Displaying the first few rows of the updated dataframe
auto_data[['engine_efficiency', 'combined_mpg', 'engine_size_squared', 'horsepower_bin', 'engine_cylinders_interaction']].head()
auto_data.drop('horsepower_bin',axis=1,inplace=True)

In [49]:
auto_data.head()

,price,highway-mpg,city-mpg,peak-rpm,horsepower,compression-ratio,stroke,bore,fuel-system,engine-size,...,num-of-doors,aspiration,fuel-type,make,normalized-losses,engine_efficiency,combined_mpg,engine_size_squared,horsepower_ordinal,engine_cylinders_interaction
0,13495.0,27,21,5000.0,111.0,9.0,2.68,3.47,mpfi,130,...,2.0,std,gas,alfa-romero,115.0,0.853846,24.6,16900,2,520
1,16500.0,27,21,5000.0,111.0,9.0,2.68,3.47,mpfi,130,...,2.0,std,gas,alfa-romero,115.0,0.853846,24.6,16900,2,520
2,16500.0,26,19,5000.0,154.0,9.0,3.47,2.68,mpfi,152,...,2.0,std,gas,alfa-romero,115.0,1.013158,23.2,23104,2,912
3,13950.0,30,24,5500.0,102.0,10.0,3.40,3.19,mpfi,109,...,4.0,std,gas,audi,164.0,0.935780,27.6,11881,2,436
4,17450.0,22,18,5500.0,115.0,8.0,3.40,3.19,mpfi,136,...,4.0,std,gas,audi,164.0,0.845588,20.4,18496,2,680


In [51]:
# Identifying categorical columns
categorical_cols = auto_data.select_dtypes(include=['object']).columns

# Apply one-hot encoding to these columns
auto_data_encoded = pd.get_dummies(auto_data, columns=categorical_cols)

# Display the shape of the new dataframe
auto_data_encoded.shape, auto_data_encoded.columns.tolist()


((201, 72),
 ['price',
  'highway-mpg',
  'city-mpg',
  'peak-rpm',
  'horsepower',
  'compression-ratio',
  'stroke',
  'bore',
  'engine-size',
  'num-of-cylinders',
  'curb-weight',
  'height',
  'width',
  'length',
  'wheel-base',
  'num-of-doors',
  'normalized-losses',
  'engine_efficiency',
  'combined_mpg',
  'engine_size_squared',
  'horsepower_ordinal',
  'engine_cylinders_interaction',
  'fuel-system_1bbl',
  'fuel-system_2bbl',
  'fuel-system_4bbl',
  'fuel-system_idi',
  'fuel-system_mfi',
  'fuel-system_mpfi',
  'fuel-system_spdi',
  'fuel-system_spfi',
  'engine-type_dohc',
  'engine-type_l',
  'engine-type_ohc',
  'engine-type_ohcf',
  'engine-type_ohcv',
  'engine-type_rotor',
  'engine-location_front',
  'engine-location_rear',
  'drive-wheels_4wd',
  'drive-wheels_fwd',
  'drive-wheels_rwd',
  'body-style_convertible',
  'body-style_hardtop',
  'body-style_hatchback',
  'body-style_sedan',
  'body-style_wagon',
  'aspiration_std',
  'aspiration_turbo',
  'fuel-type_

In [52]:
auto_data_encoded

,price,highway-mpg,city-mpg,peak-rpm,horsepower,compression-ratio,stroke,bore,engine-size,num-of-cylinders,...,make_nissan,make_peugot,make_plymouth,make_porsche,make_renault,make_saab,make_subaru,make_toyota,make_volkswagen,make_volvo
0,13495.0,27,21,5000.0,111.0,9.0,2.68,3.47,130,4,...,False,False,False,False,False,False,False,False,False,False
1,16500.0,27,21,5000.0,111.0,9.0,2.68,3.47,130,4,...,False,False,False,False,False,False,False,False,False,False
2,16500.0,26,19,5000.0,154.0,9.0,3.47,2.68,152,6,...,False,False,False,False,False,False,False,False,False,False
3,13950.0,30,24,5500.0,102.0,10.0,3.40,3.19,109,4,...,False,False,False,False,False,False,False,False,False,False
4,17450.0,22,18,5500.0,115.0,8.0,3.40,3.19,136,5,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,16845.0,28,23,5400.0,114.0,9.5,3.15,3.78,141,4,...,False,False,False,False,False,False,False,False,False,True
201,19045.0,25,19,5300.0,160.0,8.7,3.15,3.78,141,4,...,False,False,False,False,False,False,False,False,False,True
202,21485.0,23,18,5500.0,134.0,8.8,2.87,3.58,173,6,...,False,False,False,False,False,False,False,False,False,True
203,22470.0,27,26,4800.0,106.0,23.0,3.40,3.01,145,6,...,False,False,False,False,False,False,False,False,False,True


In [53]:
### saving encoded dataframe for ML Modeling.
auto_data_encoded.to_csv("data/auto_data_encoded.csv")